<a href="https://colab.research.google.com/github/wakii/ds_parrot/blob/master/resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from keras_preprocessing import image

from keras import models, layers
from keras import Input
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers, initializers, regularizers, metrics
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import BatchNormalization, Conv2D, Activation, Dense, GlobalAveragePooling2D, MaxPooling2D, ZeroPadding2D, Add

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


In [0]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 14450359007579181396, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 17588707683274577713
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 9824938068215257347
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 7304675328
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 10933863668377230563
 physical_device_desc: "device: 0, name: Tesla P4, pci bus id: 0000:00:04.0, compute capability: 6.1"]

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
root_dir = '/content/drive/My Drive'
# base_dir = root_dir + ''
base_dir = root_dir + '/2020 Spring/Parrot/CV_competition'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os

data_dir = base_dir + "/2020_parrot_dataset/"

In [0]:
from keras import models, layers
from keras import Input
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers, initializers, regularizers, metrics
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import BatchNormalization, Conv2D, Activation, Dense, GlobalAveragePooling2D, MaxPooling2D, ZeroPadding2D, Add
 
import os
import matplotlib.pyplot as plt
import numpy as np
import math
 
 
 
train_datagen=ImageDataGenerator(rescale=1./255,
                                  rotation_range=50,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range = 0.2,
                                  zoom_range = [0.2,1.0],
                                  brightness_range=[0.2,1.0],
                                  horizontal_flip=True,                                  
                                  validation_split=0.2,
                                  fill_mode='nearest')
 

 
test_datagen = ImageDataGenerator(validation_split=0.2,rescale=1./255) 
 

train_generator=train_datagen.flow_from_directory(data_dir+"/train",
                                                   target_size=(224,224),
                                                    batch_size=64,
                                                   color_mode='rgb',
                                                    class_mode='categorical',
                                                    shuffle=True,
                                                    subset= 'training')      
valid_generator = test_datagen.flow_from_directory(data_dir+"/train",  
    target_size = (224,224),
    batch_size = 64,
    shuffle = True,
    subset = 'validation',
    class_mode = 'categorical')


Found 7811 images belonging to 8 classes.
Found 1948 images belonging to 8 classes.


In [0]:
from keras.applications import resnet50

resnet_pretrained = resnet50.ResNet50(weights='imagenet')

102858752/102853048 [==============================] - 9s 0us/step


In [0]:
resnet_pretrained.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [0]:
def identity_block(X, filters):
  X_shortcut = X
  f1, f2, f3 = filters

  # First layer
  X = Conv2D(filters = f1, kernel_size=(1,1), strides = (1,1), padding='valid')(X)
  X = BatchNormalization()(X)
  X = Activation('relu')(X)

  #Second Layer
  X = Conv2D(filters = f2, kernel_size=(3,3), strides = (1,1), padding='same')(X)
  X = BatchNormalization()(X)
  X = Activation('relu')(X)

  #Third Layer
  X = Conv2D(filters = f3, kernel_size=(1,1), strides=(1,1), padding = 'valid')(X)
  X = BatchNormalization()(X)

  #Merge
  X = Add()([X,X_shortcut])
  X = Activation('relu')(X)

  return X



In [0]:
def convolutional_block(X, filters, s=2):
  f1, f2, f3 = filters
  X_shortcut = X

  X = Conv2D(f1, (1, 1), strides=(s, s), padding='valid')(X)
  X = BatchNormalization()(X)
  X = Activation('relu')(X)

  X = Conv2D(f2, kernel_size=(3,3), strides = (1,1), padding='same')(X)
  X = BatchNormalization()(X)
  X = Activation('relu')(X)


  X = Conv2D(f3, (1, 1), strides=(1, 1), padding='valid')(X)
  X_shortcut = Conv2D(f3, (1, 1), strides=(s, s), padding='valid')(X_shortcut)            
  X = BatchNormalization()(X)
  X_shortcut = BatchNormalization()(X_shortcut)

  X = Add()([X, X_shortcut])
  X = Activation('relu')(X)
  return X

In [0]:
def ResNet50(input_shape = (224,224,3), classes = 8):
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(shape=(224,224,3), dtype='float32', name='input')

    
    # Zero-Padding
    X = ZeroPadding2D(padding=(3,3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2,2))(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    X = ZeroPadding2D(padding=(1,1))(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, filters = [64, 64, 256], s=1)
    X = identity_block(X, [64, 64, 256])
    X = identity_block(X, [64, 64, 256])

    # Stage 3
    X = convolutional_block(X, filters=[128, 128, 512], s=2)
    X = identity_block(X, [128, 128, 512])
    X = identity_block(X, [128, 128, 512])
    X = identity_block(X, [128, 128, 512])

    # Stage 4
    X = convolutional_block(X, filters=[256, 256, 1024],s=2)
    X = identity_block(X,  [256, 256, 1024])
    X = identity_block(X,  [256, 256, 1024])
    X = identity_block(X, [256, 256, 1024])
    X = identity_block(X,  [256, 256, 1024])
    X = identity_block(X, [256, 256, 1024])

    # Stage 5
    X = convolutional_block(X, filters=[512, 512, 2048],s=2)
    X = identity_block(X, [512, 512, 2048])
    X = identity_block(X, [512, 512, 2048])

    # AVGPOOL
    # X = AveragePooling2D(pool_size=(2,2), padding='same')(X)
    X = GlobalAveragePooling2D()(X)
    
    # Output layer
    # X = Flatten()(X)
    X = Dense(classes, activation='softmax')(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X)

    return model

In [24]:
res = ResNet50(input_shape = (224,224,3), classes =8)
res.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

res.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_7 (ZeroPadding2D (None, 230, 230, 3)  0           input[0][0]                      
__________________________________________________________________________________________________
conv2d_61 (Conv2D)              (None, 112, 112, 64) 9472        zero_padding2d_7[0][0]           
__________________________________________________________________________________________________
batch_normalization_60 (BatchNo (None, 112, 112, 64) 256         conv2d_61[0][0]                  
____________________________________________________________________________________________